# Document Classification

This notebook provides an introduction on using NLTK and Scikit-Learn for performing Document Classification

## Initialize NTLK

Download some of the resources that NLTK needs

In [ ]:
import nltk
nltk.download('book')

## Import the additional modules

The `numpy` module is used for simple numerical operations. While it may seem odd to install a package to do simple stuff, Scikit-Learn depends on numpy anyway.

While NLTK also contains implementation of machine learning algorithms, Scikit-Learn provides general purpose machine learning implementations. Learning to use its modules meaning learning to use them on data outside NLP

In [ ]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
)

## Document Classification

Given several news articles, a model is to be trained that will identify the section of the news paper the article belongs to. For simplicity and speed, only a subset of the dataset will be used

### Loading the data

Scikit-Learn provides the `fetch_20newsgroups` which allows us to fetch news articles belonging into the categories specified in the function call.

The function returns a dictionary with different information. The `data` contains the list of documents while `target` contains the integer indicator for the labels. There are other contents of the dictionary but these are the ones needed for this example.

In [ ]:
categories = [
    'alt.atheism',
    'misc.forsale',
    'rec.sport.baseball',
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
]

In [ ]:
news = fetch_20newsgroups(subset='test', categories=categories)

In [ ]:
documents = news['data']
labels = news['target']

len(documents)

### Balancing the Dataset

While the labels are not perfectly equal, the magnitudes of the difference are not that far apart. The number of the data for each label will be as it is.

In [ ]:
np.unique(labels, return_counts=True)

### Feature Engineering

The document will be converted into word tokens for the bag of words processing. Case folding is also used given that the goal of the example is to identify topics

In [ ]:
tokenized_docs = [nltk.word_tokenize(d.lower()) for d in documents]

### Splitting the Dataset

The dataset will be split into 3 different sets, the training set, the validation set, and the testing set.

*   Train (70%): Used for training the machine learning model
*   Validation (10%): Used for evaluation of pipeline changes such as feature engineering and model hyperparameters
*   Test (20%): Used to evaluate the whole pipeline.

The scikit-learn `train_test_split` only splits in two sets. Thus, it will be used twice. The `strafity` options makes sures that the split will consider the balance of the labels while `random_state` sets the seed to allow replicability

In [ ]:
train_docs, test_docs, train_labels, test_labels = (
    train_test_split(tokenized_docs, labels, test_size=0.2, stratify=labels, random_state=0)
)

train_docs, val_docs, train_labels, val_labels = (
    train_test_split(train_docs, train_labels, test_size=1/8, stratify=train_labels, random_state=0)
)

len(train_docs), len(test_docs), len(val_docs)

### Tokens to Counts

Scikit-Learn provides a `CountVectorizer` that converts a document into the a vector of token counts. The `CountVectorizer` provides its own implementation of text proprocessing and tokenization. However, since the data are already tokenized, the `tokenizer` and `preprocessor` argument is set to a function that does nothing. The `max_features` is an optional setting defined if only the top N words are to be considered.

Take note that on the training the `fit_transform` method is used. This means that only the vocabularies in the training will be considerd in counting. The validation and test only used the `transform` method.

In [ ]:
cv = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    max_features=2000,
)

cv.fit(train_docs)

In [ ]:
{k: v for (k, v), _ in zip(cv.vocabulary_.items(), range(20))}

In [ ]:
train_cvectors = cv.transform(train_docs)
val_cvectors = cv.transform(val_docs)
test_cvectors = cv.transform(test_docs)

In [ ]:
train_cvectors.toarray()

### Counts to TF-IDF

To normalize the word counts by the inverse document frequency, the `TfidfTransformer` can be used to transform a word count matrix to the normalized TF-IDF values

Take note that on the training the `fit_transform` method is used. This means that only the IDF in the training will be used for normalization. The validation and test only used the `transform` method.

In [ ]:
tfidf = TfidfTransformer()
tfidf.fit(train_cvectors)

In [ ]:
train_vectors = tfidf.transform(train_cvectors)
val_vectors = tfidf.transform(val_cvectors)
test_vectors = tfidf.transform(test_cvectors)

In [ ]:
train_vectors.toarray()

### Random Forest

A `RandomForestClassifier` is used to classify the training data with its label. The `n_estimators` defines the number of decision trees used in the ensemble

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
rf.fit(train_vectors, train_labels)

In [ ]:
train_predict = rf.predict(train_vectors)
val_predict = rf.predict(val_vectors)
test_predict =  rf.predict(test_vectors)

### Evaluation

The different sets will be evaluated to show how the performance degrades from the training data and the out of sample data. The following metrics are computed for each set.

*   Precision
*   Recall
*   F1-Score
*   Accuracy
*   Confusion Matrix

#### Training Performance

In [ ]:
print(classification_report(train_labels, train_predict))

In [ ]:
confusion_matrix(train_labels, train_predict)

#### Validation Performance

In [ ]:
print(classification_report(val_labels, val_predict))

In [ ]:
confusion_matrix(val_labels, val_predict)

#### Test Performance

In [ ]:
print(classification_report(test_labels, test_predict))

In [ ]:
confusion_matrix(test_labels, test_predict)

### Interpretability

Random forests are also interpretable machine learning models. Random forests provides a feature importance, providing a value in how much the feature is used in separating the classes.

`CountVectorizer` creates the vector based on the sorted vocabulary, thus the feature importance are defined in the same order.

Some of the words in the top importance are directly related to the category like `sale`, `space`, `baseball`, `game` and `god`. On the other hand, words in the low importances are generic words like `proper`, `beautiful`, `results` and some rare looking words.

In [ ]:
importance_idx = rf.feature_importances_.argsort()

In [ ]:
vocabulary = sorted(cv.vocabulary_)

In [ ]:
top_importance = [vocabulary[idx] for idx in importance_idx[-10:][::-1]]
top_importance

In [ ]:
low_importance = [vocabulary[idx] for idx in importance_idx[:10]]
low_importance